In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from transformers import XLNetTokenizer, XLNetForSequenceClassification
from transformers import AutoTokenizer, OpenAIGPTForSequenceClassification
from transformers import AutoTokenizer, DebertaForSequenceClassification
from transformers import AutoTokenizer, RobertaPreLayerNormForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertConfig
import os

c:\Users\omar_\anaconda3\envs\env_cuda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch

# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # Get the number of available GPUs
    num_gpus = torch.cuda.device_count()

    # List the available GPUs
    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")
else:
    print("No GPU devices are available.")


GPU 0: NVIDIA GeForce RTX 3060
GPU 1: NVIDIA GeForce RTX 3060


In [3]:
# Specify the GPU devices you want to use
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"  # Use GPU devices 0 and 1

In [4]:
# Load the training dataset from local CSV file
train_df = pd.read_csv("train.csv")
queries = train_df['dialog'].tolist()
labels = train_df['intent'].tolist()
# added later
unique_intents = train_df['intent'].unique()
intent_to_label = {intent: label for label, intent in enumerate(unique_intents)}

# Load the inference dataset from local CSV file
test_file_path = "test.csv"

# Load test queries from the file
with open(test_file_path, "r") as file:
    inference_queries = [line.strip() for line in file]

# Define the number of intent classes
num_classes = len(train_df['intent'].unique())
print(num_classes)
print(intent_to_label)

150
{'translate': 0, 'transfer': 1, 'timer': 2, 'definition': 3, 'meaning_of_life': 4, 'insurance_change': 5, 'find_phone': 6, 'travel_alert': 7, 'pto_request': 8, 'improve_credit_score': 9, 'fun_fact': 10, 'change_language': 11, 'payday': 12, 'replacement_card_duration': 13, 'time': 14, 'application_status': 15, 'flight_status': 16, 'flip_coin': 17, 'change_user_name': 18, 'where_are_you_from': 19, 'shopping_list_update': 20, 'what_can_i_ask_you': 21, 'maybe': 22, 'oil_change_how': 23, 'restaurant_reservation': 24, 'balance': 25, 'confirm_reservation': 26, 'freeze_account': 27, 'rollover_401k': 28, 'who_made_you': 29, 'distance': 30, 'user_name': 31, 'timezone': 32, 'next_song': 33, 'transactions': 34, 'restaurant_suggestion': 35, 'rewards_balance': 36, 'pay_bill': 37, 'spending_history': 38, 'pto_request_status': 39, 'credit_score': 40, 'new_card': 41, 'lost_luggage': 42, 'repeat': 43, 'mpg': 44, 'oil_change_when': 45, 'yes': 46, 'travel_suggestion': 47, 'insurance': 48, 'todo_list_u

In [5]:
#Load a pre-trained BERT model and tokenizer
# model_name = "bert-base-uncased"
# model_name = "bert-large-uncased-whole-word-masking"
# tokenizer = BertTokenizer.from_pretrained(model_name)
# model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)

model_name = "roberta-base"
# model_name = "roberta-large"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)


# Move the model to GPUs
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Encode the queries and labels using the BERT tokenizer
encoded_inputs = tokenizer(queries, padding=True, truncation=True, return_tensors="pt")
input_ids = encoded_inputs["input_ids"]
attention_mask = encoded_inputs["attention_mask"]
labels = torch.tensor([intent_to_label[label] for label in labels]).to(device)
print(labels)

tensor([  0,   0,   0,  ..., 149, 149, 149], device='cuda:0')


In [7]:
# Split the data into training and validation sets
train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_ids, labels, test_size=0.00000000001, random_state=21)
train_masks, val_masks, _, _ = train_test_split(attention_mask, input_ids, test_size=0.00000000001, random_state=21)

In [8]:
# Create DataLoaders
batch_size = 760

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)


In [9]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


In [10]:
# Fine-tune the BERT model
optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8)
num_epochs = 64

# Wrap the model with DataParallel to use multiple GPUs
model = torch.nn.DataParallel(model)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch + 1}"):
        inputs, masks, labels = batch
        optimizer.zero_grad()

        outputs = model(inputs, attention_mask=masks, labels=labels)

        # Get the per-GPU loss
        loss = outputs.loss

        # Reduce the per-GPU loss to a scalar
        if isinstance(loss, torch.Tensor):
            loss = loss.mean()  # Average the loss if it's a tensor

        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Average training loss for epoch {epoch + 1}: {avg_train_loss}")

Epoch 1:   0%|          | 0/15 [00:00<?, ?it/s]c:\Users\omar_\anaconda3\envs\env_cuda\lib\site-packages\torch\nn\parallel\_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
c:\Users\omar_\anaconda3\envs\env_cuda\lib\site-packages\torch\cuda\nccl.py:15: UserWarning: PyTorch is not compiled with NCCL support
  warnings.warn("PyTorch is not compiled with NCCL support")
Epoch 1: 100%|██████████| 15/15 [00:19<00:00,  1.32s/it]


Average training loss for epoch 1: 5.005773671468099


Epoch 2: 100%|██████████| 15/15 [00:17<00:00,  1.15s/it]


Average training loss for epoch 2: 4.786820793151856


Epoch 3: 100%|██████████| 15/15 [00:17<00:00,  1.16s/it]


...

Average training loss for epoch 62: 0.027244438603520393


Epoch 63: 100%|██████████| 15/15 [00:17<00:00,  1.16s/it]


Average training loss for epoch 63: 0.026291109745701154


Epoch 64: 100%|██████████| 15/15 [00:17<00:00,  1.16s/it]

Average training loss for epoch 64: 0.025195595994591712


In [11]:
# Save or serialize the fine-tuned model for inference
model.module.save_pretrained("saved_model_acc_97.02_epoch64_batch_760_roberta_random21_lr-1e5")

In [12]:
# Evaluate the model on the validation set
model.module.eval()
val_predictions = []
val_true_labels = []

for batch in tqdm(val_dataloader, desc="Evaluating"):
    inputs, masks, labels = batch
    with torch.no_grad():
        outputs = model(inputs, attention_mask=masks)
    logits = outputs.logits.to("cpu")  # Move the logits to CPU
    predictions = torch.argmax(logits, dim=1)  # Use torch.argmax instead of np.argmax
    val_predictions.extend(predictions.tolist())
    val_true_labels.extend(labels.to("cpu").tolist())  # Move labels to CPU

accuracy = accuracy_score(val_true_labels, val_predictions)
classification_rep = classification_report(val_true_labels, val_predictions)

print(f"Validation accuracy: {accuracy}")
print("Classification report:\n", classification_rep)


Evaluating: 100%|██████████| 1/1 [00:00<00:00, 95.16it/s]

Validation accuracy: 1.0
Classification report:
               precision    recall  f1-score   support

          91       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1



In [13]:
# Use the fine-tuned model for inference
with open("asnwer.txt", "w") as file:
    for query in inference_queries:
        encoded_input = tokenizer(query, padding=True, truncation=True, return_tensors="pt")
        input_ids = encoded_input["input_ids"]
        attention_mask = encoded_input["attention_mask"]

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)

        logits = outputs.logits
        predicted_intent_id = torch.argmax(logits, dim=1).item()
        predicted_intent = train_df['intent'].unique()[predicted_intent_id]

        # Write the predicted intent label to the text file
        file.write(f"{predicted_intent}\n")